In [ ]:
%%html
<style>
    /* identifier les reponse aux questions */
    .comment {
        color:blue;
    }
    
    /* utilitaire pour faire des colonnes */
    .column {
        float:left;
        width:50%;
    }
</style>

# Dataset
[Plant Village](https://data.mendeley.com/datasets/tywbtsjrjv/1)

## Description

In this data-set, 39 different classes of plant leaf and background images are available. The data-set containing 61,486 images. We used six different augmentation techniques for increasing the data-set size. The techniques are image flipping, Gamma correction, noise injection, PCA color augmentation, rotation, and Scaling.

The classes are: 

<br>

<div class="column">

1. Apple_scab
2. Apple_black_rot
3. Apple_cedar_apple_rust
4. Apple_healthy
5. Background_without_leaves
6. Blueberry_healthy
7. Cherry_powdery_mildew
8. Cherry_healthy
9. Corn_gray_leaf_spot
10. Corn_common_rust
11. Corn_northern_leaf_blight
12. Corn_healthy
13. Grape_black_rot
14. Grape_black_measles
15. Grape_leaf_blight
16. Grape_healthy
17. Orange_haunglongbing
18. Peach_bacterial_spot
19. Peach_healthy
20. Pepper_bacterial_spot

</div>

<div class="column">
    
21. Pepper_healthy
22. Potato_early_blight
23. Potato_healthy
24. Potato_late_blight
25. Raspberry_healthy
26. Soybean_healthy
27. Squash_powdery_mildew
28. Strawberry_healthy
29. Strawberry_leaf_scorch
30. Tomato_bacterial_spot
31. Tomato_early_blight
32. Tomato_healthy
33. Tomato_late_blight
34. Tomato_leaf_mold
35. Tomato_septoria_leaf_spot
36. Tomato_spider_mites_two-spotted_spider_mite
37. Tomato_target_spot
38. Tomato_mosaic_virus
39. Tomato_yellow_leaf_curl_virus
    
</div>

In [1]:
%matplotlib inline

#
# import utilitaires
#
import cv2
import numpy as np
import pandas as pd
import pickle
import random
import seaborn as sns

from dataset_helpers import dataset_load_config, dataset_load
from jupyter_helpers import display_html
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# make random deterministic
random.seed(42)

In [2]:
#
# retreive dataset PlantVillage
# does necessary work to get final .csv
# some preliminary cleanup is done
#
dataset_config = dataset_load_config("PlantVillage_Config.json")
if dataset_config is None:
    display_html(f"<b>Invalid dataset config</b>")

In [3]:
plantvillage = dataset_load(dataset_config.dataset, 
                            keep_download=True,
                            force_thumbnail_generation=True)
if plantvillage is None:
    display_html(f"<b>Invalid dataset</b>")

  0%|          

Apple___Apple_scab
/images/Apple___Apple_scab
['', 'images', 'Apple___Apple_scab']
['image (1).JPG', 'image (10).JPG', 'image (100).JPG', 'image (1000).JPG', 'image (101).JPG', 'image (102).JPG', 'image (103).JPG', 'image (104).JPG', 'image (105).JPG', 'image (106).JPG', 'image (107).JPG', 'image (108).JPG', 'image (109).JPG', 'image (11).JPG', 'image (110).JPG', 'image (111).JPG', 'image (112).JPG', 'image (113).JPG', 'image (114).JPG', 'image (115).JPG', 'image (116).JPG', 'image (117).JPG', 'image (118).JPG', 'image (119).JPG', 'image (12).JPG', 'image (120).JPG', 'image (121).JPG', 'image (122).JPG', 'image (123).JPG', 'image (124).JPG', 'image (125).JPG', 'image (126).JPG', 'image (127).JPG', 'image (128).JPG', 'image (129).JPG', 'image (13).JPG', 'image (130).JPG', 'image (131).JPG', 'image (132).JPG', 'image (133).JPG', 'image (134).JPG', 'image (135).JPG', 'image (136).JPG', 'image (137).JPG', 'image (138).JPG', 'image (139).JPG', 'image (14).JPG', 'image (140).JPG', 'image (14

TypeError: Accessing a group is done with bytes or str,  not <class 'ellipsis'>

# Exploration

In [ ]:
display_html(f"<b>{dataset_config.preprocess_path}</b> - data {plantvillage.shape}")
display(plantvillage.head(5))

In [ ]:
display_html(f"<b>{dataset_config.preprocess_path}</b> - info")
plantvillage.info()

In [ ]:
species = plantvillage["species"].unique()
species.sort()
display_html(f"<b>Modalites de 'species'</b> - count: {species.size}")
display(list(species))

In [ ]:
labels = plantvillage["label"].unique()
labels.sort()
display_html(f"<b>Modalites de 'label'</b> - count: {len(labels)}")
display(list(labels))

<font class="comment">

**Notes**
* Background_without_leaves a ete intentionelment enleve
* les labels semblent avoir evolues par rapport a la liste cite ci-haut

In [ ]:
#
# distribution de label
#
FIGSIZE = (10, 4)

disease_count = plantvillage["label"].value_counts()
disease_count.sort_values(inplace=True, ascending=False)

plt.figure(figsize=FIGSIZE)
disease_count.plot.bar(title="Effectifs de 'label'",
                       ylabel="count")
plt.show()

In [ ]:
#
# relation species/label
#
doi = plantvillage[["label", "species"]]

disease_samples_per_species = doi.groupby("species").count()
disease_samples_per_species.sort_values("label", inplace=True, ascending=False)

In [ ]:
healthy_crit = doi["label"].str.contains("healthy")

healthy = doi[healthy_crit]
healthy = healthy \
            .groupby("species") \
            .count() \
            .rename(columns={"label": "healthy"})

not_healthy = doi[ ~healthy_crit ]
not_healthy = not_healthy \
                .groupby("species") \
                .count() \
                .rename(columns={"label": "autres"})

healthy_vs_other_per_species = pd.merge(healthy, 
                                        not_healthy,
                                        how="outer",
                                        on="species")

# for ease of comparison, keep same order 
# as disease_samples_per_species
healthy_vs_other_per_species = healthy_vs_other_per_species \
                                   .reindex(disease_samples_per_species \
                                                .index.to_list())

In [ ]:
fig = plt.figure(figsize=FIGSIZE)
fig.suptitle("Effectifs de 'label' par espece de plante")

# plt.subplot(121)
# disease_samples_per_species.plot.bar(ax=plt.gca(),
#                                      xlabel="",
#                                      ylabel="count",
#                                      legend=False)

plt.subplot(121)
healthy_vs_other_per_species.plot.bar(ax=plt.gca(),
                                      stacked=True, 
                                      xlabel="",
                                      ylabel="count")
plt.title("'healthy' vs autres", y=1.025)

plt.subplot(122)
_, _, pcts = plt.pie(x=disease_samples_per_species.values.flatten(),
                     labels=disease_samples_per_species.index.values,
                     autopct='%.0f%%',
                     radius=1.25,
                     wedgeprops={'linewidth': 1.0, 
                                 'edgecolor': 'white'})
plt.title("Toute maladies confondues", y=1.025)
plt.setp(pcts, color='white')

plt.tight_layout()
plt.show()

In [ ]:
def load_image(filename):
    image = cv2.imread(filename)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
#
# visualize some images
#
SAMPLEs_GRID = (2, 7)

# select some random samples
sample_indices = random.sample(range(plantvillage.shape[0]), SAMPLEs_GRID[0] * SAMPLEs_GRID[1])
sample_indices.sort() 

samples = plantvillage.loc[sample_indices].copy()
samples["thumbnail"] = samples["thumbnail_path"].apply(load_image)

fig = plt.figure(figsize=FIGSIZE)
fig.suptitle("Echantillons maladies")

for i, (_, sample) in enumerate(samples.iterrows()):
    plt.subplot(SAMPLEs_GRID[0], SAMPLEs_GRID[1], i + 1)
    plt.axis("off")
    
    # adjust title: some disease have very long name
    species = sample['species']
    disease = sample['disease'].replace(species, "")
    disease = disease.split("_") 
    count = len(disease) 
    if count > 2:
        count //= 2
        disease0 = " ".join(disease[:count])
        disease1 = " ".join(disease[count:])
        disease = f"{disease0}\n{disease1}"
    else:
        disease = " ".join(disease)
    
    plt.title(f"{species}\n{disease}")    
    plt.imshow(sample["thumbnail"])
    plt.gca().title.set_size(10)

plt.tight_layout()
fig.subplots_adjust(hspace=0.5, top=0.8)
plt.show()

# Modelisation

Mettre un peu de bla bla pour justifier ce qu'on fait, citer les papiers et tout

**TODO**
* Train/test split
* Pipeline de traitement - Bag of Visual Words
    * Rescale en 256x256?
    * Remapper 0/1 et float
    * Threshold background/foreground    
    * Slicer les images
    * Detecter les features via SIFT/ORB
    * Passer dans un KMean
    * Mettre le tout dans notre dictionnaire

* Passer test data dans pipeline
    * Sparse vector pour une image non connue
    * Identifier avec K Nearest Neighboor

In [ ]:
#
# split train/test
#
train, test = train_test_split(plantvillage, 
                               train_size=0.8, 
                               random_state=42, 
                               stratify=plantvillage["label"])

In [ ]:
# validation split size
display_html(f"<b>Dataset:</b> <font style='color:red;'>{plantvillage.size}</font>, "
             f"<b>Train:</b> <font style='color:red;'>{train.size}</font>, "
             f"<b>Test:</b> <font style='color:red;'>{test.size}</font> "
              "- 80% / 20% split")

In [ ]:
# validation split distribution
size = plantvillage.shape[0]
frequencies = pd.DataFrame({"dataset": disease_count / size,
                            "train": train["label"].value_counts() / size,
                            "test": test["label"].value_counts() / size})
frequencies = frequencies.reindex(index=disease_count.index.to_list())

plt.figure(figsize=FIGSIZE)
frequencies.plot.bar(ax=plt.gca(),
                     stacked=True,
                     title="Distribution 'label' dataset vs train vs test\n"
                           "(couleurs meme taille dans chaque bar est mieux)",
                     ylabel="%")
plt.show()

In [ ]:
#
# construire les features
#
desc_factory = cv2.ORB_create()
feature_list = np.empty((0, 32), dtype=np.uint8)

progress = tqdm(iterable=plantvillage.iterrows(),
                total=plantvillage.shape[0],
                bar_format="{l_bar}{bar}{postfix}")

for index, row in progress:
    image_path = row["image_path"]
    progress.set_postfix_str(image_path)
    
    image = load_image(image_path)

    # tiling irait ici
    # detection backgroud irait ici
    _, descs = desc_factory.detectAndCompute(image, None)
    if descs is None:
        continue
    
    feature_list = np.append(feature_list, descs, axis=0)
    
    del image
    
#     if index > 100:
#         break

with open('descs.pkl', 'wb') as f:
    pickle.dump(list(feature_list), f)
    
# save the data to hdf5
print(plantvillage.shape[0])
print(feature_list.shape)

# Evaluation

* Entrainement avec les stats